# Advanced Composition

This tutorial demonstrates advanced patterns for composing complex prompts from modular pieces.

Learn how to:
- Build reusable prompt libraries
- Create deeply nested structures
- Compose prompts conditionally
- Design flexible prompt templates

In [ ]:
import json

from t_prompts import prompt

## Building Prompt Libraries

Create reusable prompt components that can be composed together.

In [ ]:
# Create a library of system prompts
SYSTEM_PROMPTS = {
    "helpful": prompt(t"{'You are a helpful assistant.':system}"),
    "concise": prompt(t"{'Be concise and direct.':system}"),
    "polite": prompt(t"{'Always be polite and respectful.':system}"),
}

# Use them in different contexts
query = "What is Python?"
full = prompt(t"{SYSTEM_PROMPTS['helpful']:sys}\n\nUser: {query:q}")
print(str(full))

## Multi-Level Nesting

Build deeply nested structures for organizing complex prompts.

In [ ]:
# Level 1: Basic facts
name = "Alice"
age = "30"
person = prompt(t"Name: {name:n}, Age: {age:a}")

# Level 2: Context
city = "Paris"
context = prompt(t"{person:person} lives in {city:c}")

# Level 3: Full prompt
task = "Write a short bio"
full = prompt(t"Task: {task:t}\nContext: {context:ctx}")

print(str(full))
print("\nNavigate nested structure:")
print(f"  Name: {full['ctx']['person']['n']}")
print(f"  Age: {full['ctx']['person']['a']}")
print(f"  City: {full['ctx']['c']}")

## Conditional Composition

Build prompts conditionally based on runtime data.

In [ ]:
def build_prompt_with_context(query, include_examples=False, include_constraints=False):
    """Build a prompt with optional sections."""
    parts = []

    # Always include the query
    parts.append(prompt(t"Query: {query:q}"))

    # Optionally include examples
    if include_examples:
        examples = "Example: 2+2=4"
        parts.append(prompt(t"\n{examples:ex}"))

    # Optionally include constraints
    if include_constraints:
        constraints = "Keep response under 50 words"
        parts.append(prompt(t"\n{constraints:constraints}"))

    # Combine all parts
    return prompt(t"{parts:parts:sep=}")


# Try different combinations
p1 = build_prompt_with_context("What is 5+3?")
print("Minimal:")
print(str(p1))

p2 = build_prompt_with_context("What is 5+3?", include_examples=True, include_constraints=True)
print("\nWith everything:")
print(str(p2))

## Prompt Templates with Slots

Create reusable templates that can be filled in with specific values.

In [ ]:
def create_translation_prompt(text, target_lang, include_context=True):
    """Template for translation prompts."""
    task = f"Translate to {target_lang}"
    p_task = prompt(t"{task:task}")
    p_text = prompt(t"Text: {text:text}")

    if include_context:
        context = "Maintain the original tone and style."
        p_context = prompt(t"Context: {context:ctx}")
        return prompt(t"{p_task:t}\n{p_context:c}\n{p_text:txt}")
    else:
        return prompt(t"{p_task:t}\n{p_text:txt}")


# Use the template
p = create_translation_prompt("Hello, world!", "French")
print(str(p))
print(f"\nKeys: {list(p.keys())}")

## Combining Lists and Nesting

Use lists within nested prompts for maximum flexibility.

In [ ]:
# Create a list of examples
example_texts = ["Python is a language", "JavaScript runs in browsers", "Rust is memory-safe"]
examples = [prompt(t"{example_texts[i]:ex}") for i in range(len(example_texts))]

# Wrap the examples in a section
examples_section = prompt(t"Examples:\n{examples:examples}")

# Create the full prompt with the nested section
instruction = "Based on these examples, describe programming languages briefly."
full = prompt(t"{instruction:inst}\n\n{examples_section:ex_section}")

print(str(full))
print("\nAccess nested list:")
examples_node = full["ex_section"]["examples"]
print(f"Number of examples: {len(examples_node)}")
print(f"First example: {examples_node[0]['ex']}")

## Exporting Complex Structures

Export the full structure as JSON for analysis or storage.

In [ ]:
# Export the complex structure
json_data = full.toJSON()

# Pretty print
print("JSON export (truncated):")
print(json.dumps(json_data, indent=2)[:500] + "...")

print(f"\nPrompt ID: {json_data['prompt_id']}")
print(f"Number of children: {len(json_data['children'])}")

## Building Conversations

Create conversation prompts from structured message data.

In [ ]:
def build_conversation(messages):
    """Build a conversation prompt from a list of (role, content) tuples."""
    message_prompts = [prompt(t"{messages[i][0]}: {messages[i][1]: msg_{str(i)} }") for i in range(len(messages))]

    return prompt(t"{message_prompts:messages:sep=\n}")


# Build a conversation
conversation = build_conversation(
    [
        ("User", "Hello!"),
        ("Assistant", "Hi! How can I help?"),
        ("User", "What is Python?"),
        ("Assistant", "Python is a programming language."),
    ]
)

print(str(conversation))
print(f"\nMessage count: {len(conversation['messages'])}")

## Factory Functions for Prompts

Use factory functions to create consistent prompt structures.

In [ ]:
def create_qa_prompt(question, context=None, examples=None):
    """Factory for question-answering prompts."""
    parts = []

    # Add context if provided
    if context:
        parts.append(prompt(t"Context: {context:ctx}"))

    # Add examples if provided
    if examples:
        example_prompts = [prompt(t"Q: {examples[i][0]:q}\nA: {examples[i][1]:a}") for i in range(len(examples))]
        parts.append(prompt(t"\nExamples:\n{example_prompts:examples:sep=\n\n}"))

    # Add the question
    parts.append(prompt(t"\nQuestion: {question:question}"))

    return prompt(t"{parts:parts:sep=}")


# Use the factory
p = create_qa_prompt(
    question="What is 10+15?", context="Basic arithmetic", examples=[("What is 2+2?", "4"), ("What is 5+3?", "8")]
)

print(str(p))

## Hierarchical Prompt Structures

Build hierarchical structures that mirror your domain model.

In [ ]:
class PromptBuilder:
    """Builder for hierarchical prompts."""

    def __init__(self):
        self.sections = []

    def add_section(self, title, content):
        """Add a titled section."""
        section = prompt(t"{content:content:header={title}}")
        self.sections.append(section)
        return self

    def build(self):
        """Build the final prompt."""
        # Create separate section list for each build
        section_prompts = [prompt(t"{self.sections[i]:section_{str(i)}}") for i in range(len(self.sections))]
        return prompt(t"{section_prompts:sections:sep=\n\n}")


# Use the builder
builder = PromptBuilder()
builder.add_section("Overview", "This is an overview section.")
builder.add_section("Details", "Here are the details.")
builder.add_section("Conclusion", "This is the conclusion.")

p = builder.build()
print(str(p))

## Dynamic Prompt Assembly

Assemble prompts from configuration data.

In [ ]:
def assemble_from_config(config):
    """Assemble a prompt from a configuration dictionary."""
    parts = []

    # Process each section in order
    for section in config["sections"]:
        section_type = section["type"]

        if section_type == "text":
            key = section.get("key", "text")
            content = section["content"]
            parts.append(prompt(t"{content:{key}}"))

        elif section_type == "header":
            title = section["title"]
            content = section["content"]
            key = section.get("key", "section")
            parts.append(prompt(t"{content:{key}:header={title}}"))

    return prompt(t"{parts:parts:sep=\n\n}")


# Example configuration
config = {
    "sections": [
        {"type": "text", "key": "intro", "content": "Introduction to the topic."},
        {"type": "header", "key": "main", "title": "Main Content", "content": "This is the main content section."},
    ]
}

p = assemble_from_config(config)
print(str(p))

## Summary

Advanced composition patterns enable powerful prompt engineering:

✅ **Reusable libraries** - Build collections of prompt components  
✅ **Deep nesting** - Create complex hierarchical structures  
✅ **Conditional composition** - Assemble prompts based on runtime data  
✅ **Templates** - Design flexible, reusable prompt patterns  
✅ **Lists and nesting** - Combine lists with nested structures  
✅ **Factory functions** - Encapsulate prompt creation logic  
✅ **Builders** - Use builder patterns for step-by-step construction  
✅ **Configuration-driven** - Assemble from data structures  

These patterns let you build sophisticated, maintainable prompt systems that scale with your application's complexity.